In [5]:
# from hmmlearn.hmm import GaussianHMM
from hmmlearn import hmm
import numpy as np
from matplotlib import cm, pyplot as plt
import matplotlib.dates as dates
import pandas as pd
from datetime import timedelta,date

import sklearn
import warnings
warnings.filterwarnings("ignore")

def GHMM_Generate(Sec):
    
    beginDate = '2006-1-1'
    endDate = '2014-12-30'
    n_state = 4

    data_raw = get_price(Sec, start_date = beginDate, end_date=endDate, frequency='daily', fields=['close','volume','money'],fq = "pre")
    
    
    logRet_5 = np.log(np.array(data_raw['close'][5:])) - np.log(np.array(data_raw['close'][:-5]))

    logRet_20 = np.log(np.array(data_raw['close'][20:])) - np.log(np.array(data_raw['close'][:-20]))

    logVol_5 = np.log(np.array(data_raw['volume'][5:])) - np.log(np.array(data_raw['volume'][:-5]))

    logVol_20 = np.log(np.array(data_raw['volume'][20:])) - np.log(np.array(data_raw['volume'][:-20]))

    logMoney_5 = np.log(np.array(data_raw['money'][5:])) - np.log(np.array(data_raw['money'][:-5]))

    logMoney_20 = np.log(np.array(data_raw['money'][20:])) - np.log(np.array(data_raw['money'][:-20]))

    std = pd.rolling_std(data_raw['close'].pct_change(),20)

    data_len = len(data_raw['close']) - 50

    Train_Data = np.column_stack([logRet_5[-data_len:], \
                              logRet_20[-data_len:], \
                              logVol_5[-data_len:], \
                              logVol_20[-data_len:], \
                              logMoney_5[-data_len:], \
                              logMoney_20[-data_len:], \
                              std[-data_len:]])
    
    
    Date = pd.to_datetime(data_raw.index[-data_len:])
    
    model = hmm.GaussianHMM(n_components= n_state, covariance_type="full", n_iter=2000).fit(Train_Data)

    hidden_states = model.predict(Train_Data)
    
    state_pd = pd.concat([pd.DataFrame(Date[-data_len:]),pd.DataFrame(list(hidden_states[-data_len:]))],axis = 1,names = ['date','state'])
    state_pd.columns = ['date','state']

    state_choose = state_pd[(state_pd['date'] > datetime.datetime.strptime("2008-1-1", "%Y-%m-%d"))&(state_pd['date'] < datetime.datetime.strptime("2008-10-31", "%Y-%m-%d"))]['state'].value_counts()
    state_BigLoss = state_choose[state_choose == state_choose.max()].index[0]

    state_choose = state_pd[(state_pd['date'] > datetime.datetime.strptime("2007-1-1", "%Y-%m-%d"))&(state_pd['date'] < datetime.datetime.strptime("2007-10-1", "%Y-%m-%d"))]['state'].value_counts()
    state_BigBonus = state_choose[state_choose == state_choose.max()].index[0]

    state_choose = state_pd[(state_pd['date'] > datetime.datetime.strptime("2011-1-1", "%Y-%m-%d"))&(state_pd['date'] < datetime.datetime.strptime("2013-1-1", "%Y-%m-%d"))]['state'].value_counts()
    state_MinorLoss = state_choose[state_choose == state_choose.max()].index[0]

    state_choose = state_pd[(state_pd['date'] > datetime.datetime.strptime("2014-6-1", "%Y-%m-%d"))&(state_pd['date'] < datetime.datetime.strptime("2014-12-31", "%Y-%m-%d"))]['state'].value_counts()
    state_MinorBonus = state_choose[state_choose == state_choose.max()].index[0]

    return model,state_BigLoss,state_BigBonus,state_MinorLoss,state_MinorBonus



def single_state_estimation(Sec,model_read,enddate = "2018-12-20"):
    HMM_model = model_read[0]
    state_BigLoss = model_read[1]
    state_BigBonus = model_read[2]
    state_MinorLoss = model_read[3]
    state_MinorBonus =  model_read[4]

    # 生成模型对应的参数
    data_raw = get_price(Sec, count = 100, end_date=enddate, frequency='daily', fields=['close','volume','money'],fq = "pre")

    logRet_5 = np.log(np.array(data_raw['close'][5:])) - np.log(np.array(data_raw['close'][:-5]))

    logRet_20 = np.log(np.array(data_raw['close'][20:])) - np.log(np.array(data_raw['close'][:-20]))

    logVol_5 = np.log(np.array(data_raw['volume'][5:])) - np.log(np.array(data_raw['volume'][:-5]))

    logVol_20 = np.log(np.array(data_raw['volume'][20:])) - np.log(np.array(data_raw['volume'][:-20]))

    logMoney_5 = np.log(np.array(data_raw['money'][5:])) - np.log(np.array(data_raw['money'][:-5]))

    logMoney_20 = np.log(np.array(data_raw['money'][20:])) - np.log(np.array(data_raw['money'][:-20]))

    std = pd.rolling_std(data_raw['close'].pct_change(),20)

    data_len = len(data_raw['close']) - 50


    Train_Data = np.column_stack([logRet_5[-data_len:], \
                                  logRet_20[-data_len:], \
                                  logVol_5[-data_len:], \
                                  logVol_20[-data_len:], \
                                  logMoney_5[-data_len:], \
                                  logMoney_20[-data_len:], \
                                  std[-data_len:]])
    
    

    hidden_states = HMM_model.predict(Train_Data)
    current_states = hidden_states[-1]
    
    if current_states == state_BigLoss:
        return 'bl'
    elif current_states == state_BigBonus:
        return 'bb'
    elif current_states == state_MinorLoss:
        return 'ml'
    elif current_states == state_MinorBonus:
        return 'mb'
    else:
        return 0
    


In [9]:
# data_total_pd = pd.read_csv(".\\HMM\\data_total_pd.csv",index_col = 0)
data_total_pd = pd.read_csv("data_total_pd.csv",index_col = 0)

In [10]:
data_total_pd

,price_raw,price_next,price_diff,state,alpha,score,market_yield,adjust_yield
2015-01-05,3641.54,3641.06,-0.48,bb,1.0,0.0,-0.000132,0.0
2015-01-06,3641.06,3643.79,2.73,bb,1.0,0.0,0.000750,0.0
2015-01-07,3643.79,3559.26,-84.53,bb,1.0,0.0,-0.023472,0.0
2015-01-08,3559.26,3546.72,-12.54,bb,1.0,0.0,-0.003529,0.0
2015-01-09,3546.72,3513.58,-33.14,bb,1.0,0.0,-0.009388,0.0
2015-01-12,3513.58,3514.04,0.46,bb,1.0,0.0,0.000131,0.0
2015-01-13,3514.04,3502.42,-11.62,bb,1.0,0.0,-0.003312,0.0
2015-01-14,3502.42,3604.12,101.70,bb,1.0,0.0,0.028623,0.0
2015-01-15,3604.12,3635.15,31.03,bb,1.0,0.0,0.008573,0.0
2015-01-16,3635.15,3355.16,-279.99,bb,1.0,0.0,-0.080151,0.0


In [11]:
def cal_score(data_total_pd,n,score_BigBonus,score_BigLoss,score_MinorBonus,score_MinorLoss):
    # 每个日期进行循环比对
    date_operate = data_total_pd.index
    
    score_list = []
    for date_i in date_operate[:]:

        # 状态的pandas数组
        States_pd_current = data_total_pd[(data_total_pd.index <= date_i)&(data_total_pd.index > (date_i - timedelta(days = n)))]

        score_current = 0
        for index, row in States_pd_current.iterrows():
            if row['state'] == "bb":
                score_current += score_BigBonus
            elif row['state'] == "bl":
                score_current += score_BigLoss
            elif row['state'] == "mb":
                score_current += score_MinorBonus
            elif row['state'] == "ml":
                score_current += score_MinorLoss
        score_list.append(score_current)

    data_total_pd['score'] = np.array(score_list)
    return data_total_pd


In [12]:
# 转换为datetime格式
data_total_pd['date'] = data_total_pd.index
data_total_pd
data_total_pd['date'] = pd.to_datetime(data_total_pd['date'])
data_total_pd
data_total_pd.set_index('date',inplace=True)

In [ ]:
# 上限总分值：10分
total_score = 10

# 开始循环，寻找合理参数
# 存储参数的数组
para_list = []
i = 0
for score_BigLoss in range(-10,1,2):
    for score_BigBonus in range(0,10,2):
        for score_MinorLoss in range(-10,1,2):
            for score_MinorBonus in range(0,10,2):
                for n_windows in range(0,40,4):

                    # 标识目前的进度                   
                    # i += 1
                    # print "%d、正在计算：n为%d,score_BigBonus为%d,score_BigLoss为%d,score_MinorBonus为%d,score_MinorLoss为%d" \
                    #     %(i,n_windows,score_BigBonus,score_BigLoss,score_MinorBonus,score_MinorLoss)
    
                    # 列表中几个参数需要赋初值

                    # 仓位的关键参数，初始值默认为1，每次循环要重新赋值
                    alpha = np.ones(data_total_pd.shape[0])
                    data_total_pd['alpha'] = alpha

                    # 评分，评分初始值默认为0，每次循环要重新赋值
                    score = np.zeros(data_total_pd.shape[0])
                    data_total_pd['score'] = score

                    # 通过仓位调整后的收益率，每次循环要重新赋值
                    data_total_pd['adjust_yield'] = np.zeros(data_total_pd.shape[0])

                    
                    data_total_pd = cal_score(data_total_pd,n_windows,score_BigBonus,score_BigLoss,score_MinorBonus,score_MinorLoss)

                    data_total_pd['alpha'] = np.where(data_total_pd['score'] > 0,data_total_pd['score']/total_score,0)

                    data_total_pd['adjust_yield'] =  \
                            np.log( 1 + data_total_pd['alpha'] * data_total_pd['price_diff'] / data_total_pd['price_raw'])

                    
                    # 避免后续出错
                    data_total_pd.fillna(0,inplace = True)
                    
                    # 计算胜率
                    win_rate = \
                        float(sum(data_total_pd['adjust_yield'] > data_total_pd['market_yield'])) / float(len(data_total_pd['adjust_yield']))

                    if sum(data_total_pd['adjust_yield']) > sum(data_total_pd['market_yield']):
                        if win_rate > 0.5:
                            para_list.append([n_windows,score_BigBonus,score_BigLoss,score_MinorBonus,score_MinorLoss,win_rate,sum(data_total_pd['adjust_yield'])])
                            # 提示                   
                            print("找到参数组：n为%d,score_BigBonus为%d,score_BigLoss为%d,score_MinorBonus为%d,score_MinorLoss为%d，胜率为%f，收益率为%f" \
                                    %(n_windows,score_BigBonus,score_BigLoss,score_MinorBonus,score_MinorLoss,win_rate,sum(data_total_pd['adjust_yield'])))



找到参数组：n为8,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为2,score_MinorLoss为-10，胜率为0.507812，收益率为0.306833
找到参数组：n为12,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为2,score_MinorLoss为-10，胜率为0.506696，收益率为0.460412
找到参数组：n为16,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为2,score_MinorLoss为-10，胜率为0.506696，收益率为0.550978
找到参数组：n为20,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为2,score_MinorLoss为-10，胜率为0.504464，收益率为0.603997
找到参数组：n为8,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.506696，收益率为0.555246
找到参数组：n为12,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.512277，收益率为0.765971
找到参数组：n为16,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.513393，收益率为0.976143
找到参数组：n为20,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.516741，收益率为1.218963
找到参数组：n为24,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.514509，收益率为0.619636
找到参数组：n为28,s

找到参数组：n为28,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为2,score_MinorLoss为-4，胜率为0.501116，收益率为0.197834
找到参数组：n为8,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.512277，收益率为0.475674
找到参数组：n为12,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.512277，收益率为0.641830
找到参数组：n为16,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.518973，收益率为0.903019
找到参数组：n为20,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.515625，收益率为1.267171
找到参数组：n为24,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.515625，收益率为0.847944
找到参数组：n为28,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.508929，收益率为0.529774
找到参数组：n为32,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.514509，收益率为0.072718
找到参数组：n为4,score_BigBonus为0,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-4，胜率为0.518973，收益率为0.354811
找到参数组：n为8,score_BigBo

找到参数组：n为24,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为2,score_MinorLoss为-10，胜率为0.504464，收益率为0.344819
找到参数组：n为8,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.508929，收益率为0.468356
找到参数组：n为12,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.516741，收益率为0.724638
找到参数组：n为16,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.516741，收益率为1.072426
找到参数组：n为20,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.520089，收益率为1.317289
找到参数组：n为24,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.517857，收益率为0.719160
找到参数组：n为28,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.511161，收益率为0.174229
找到参数组：n为4,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-10，胜率为0.514509，收益率为0.325493
找到参数组：n为8,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-10，胜率为0.518973，收益率为0.609675
找到参数组：n为12,sc

找到参数组：n为12,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.516741，收益率为0.600497
找到参数组：n为16,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.522321，收益率为0.999302
找到参数组：n为20,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.518973，收益率为1.365497
找到参数组：n为24,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.518973，收益率为0.947467
找到参数组：n为28,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.508929，收益率为0.629456
找到参数组：n为32,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.513393，收益率为0.177040
找到参数组：n为4,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-4，胜率为0.520089，收益率为0.306898
找到参数组：n为8,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-4，胜率为0.515625，收益率为0.518893
找到参数组：n为12,score_BigBonus为2,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-4，胜率为0.525670，收益率为0.703707
找到参数组：n为16,score_BigB

找到参数组：n为12,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.514509，收益率为0.587607
找到参数组：n为16,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.514509，收益率为1.155604
找到参数组：n为20,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.520089，收益率为1.411826
找到参数组：n为24,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.515625，收益率为0.762750
找到参数组：n为28,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-10，胜率为0.510045，收益率为0.182141
找到参数组：n为4,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-10，胜率为0.512277，收益率为0.256110
找到参数组：n为8,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-10，胜率为0.517857，收益率为0.447548
找到参数组：n为12,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-10，胜率为0.517857，收益率为0.781639
找到参数组：n为16,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-10，胜率为0.525670，收益率为1.298667
找到参数组：n为20,s

找到参数组：n为20,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.518973，收益率为1.460034
找到参数组：n为24,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.516741，收益率为0.991056
找到参数组：n为28,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.507812，收益率为0.637149
找到参数组：n为32,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-4，胜率为0.515625，收益率为0.095133
找到参数组：n为4,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-4，胜率为0.517857，收益率为0.237514
找到参数组：n为8,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-4，胜率为0.514509，收益率为0.356766
找到参数组：n为12,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-4，胜率为0.522321，收益率为0.558984
找到参数组：n为16,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-4，胜率为0.516741，收益率为1.043637
找到参数组：n为20,score_BigBonus为4,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-4，胜率为0.514509，收益率为1.431398
找到参数组：n为24,score_BigB

找到参数组：n为4,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-10，胜率为0.512277，收益率为0.151174
找到参数组：n为8,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-10，胜率为0.518973，收益率为0.299561
找到参数组：n为12,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-10，胜率为0.520089，收益率为0.584782
找到参数组：n为16,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-10，胜率为0.527902，收益率为1.190231
找到参数组：n为20,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-10，胜率为0.518973，收益率为1.492721
找到参数组：n为24,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为6,score_MinorLoss为-10，胜率为0.518973，收益率为0.439779
找到参数组：n为4,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为8,score_MinorLoss为-10，胜率为0.520089，收益率为0.223321
找到参数组：n为8,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为8,score_MinorLoss为-10，胜率为0.515625，收益率为0.348996
找到参数组：n为12,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为8,score_MinorLoss为-10，胜率为0.516741，收益率为0.615721
找到参数组：n为16,sco

找到参数组：n为8,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为8,score_MinorLoss为-4，胜率为0.523438，收益率为0.249953
找到参数组：n为12,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为8,score_MinorLoss为-4，胜率为0.517857，收益率为0.237438
找到参数组：n为16,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为8,score_MinorLoss为-4，胜率为0.513393，收益率为0.473403
找到参数组：n为20,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为8,score_MinorLoss为-4，胜率为0.512277，收益率为0.690543
找到参数组：n为32,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为8,score_MinorLoss为-4，胜率为0.526786，收益率为0.244813
找到参数组：n为12,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为2,score_MinorLoss为-2，胜率为0.505580，收益率为-0.031446
找到参数组：n为16,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为2,score_MinorLoss为-2，胜率为0.517857，收益率为0.669858
找到参数组：n为20,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为2,score_MinorLoss为-2，胜率为0.522321，收益率为0.912281
找到参数组：n为24,score_BigBonus为6,score_BigLoss为-10,score_MinorBonus为2,score_MinorLoss为-2，胜率为0.515625，收益率为0.605830
找到参数组：n为28,score_Bi

找到参数组：n为16,score_BigBonus为8,score_BigLoss为-10,score_MinorBonus为8,score_MinorLoss为-8，胜率为0.516741，收益率为0.481244
找到参数组：n为20,score_BigBonus为8,score_BigLoss为-10,score_MinorBonus为8,score_MinorLoss为-8，胜率为0.521205，收益率为0.476302
找到参数组：n为32,score_BigBonus为8,score_BigLoss为-10,score_MinorBonus为8,score_MinorLoss为-8，胜率为0.517857，收益率为0.029166
找到参数组：n为16,score_BigBonus为8,score_BigLoss为-10,score_MinorBonus为2,score_MinorLoss为-6，胜率为0.513393，收益率为0.389130
找到参数组：n为20,score_BigBonus为8,score_BigLoss为-10,score_MinorBonus为2,score_MinorLoss为-6，胜率为0.511161，收益率为0.357155
找到参数组：n为8,score_BigBonus为8,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-6，胜率为0.514509，收益率为-0.055314
找到参数组：n为12,score_BigBonus为8,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-6，胜率为0.515625，收益率为0.002967
找到参数组：n为16,score_BigBonus为8,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-6，胜率为0.524554，收益率为0.734822
找到参数组：n为20,score_BigBonus为8,score_BigLoss为-10,score_MinorBonus为4,score_MinorLoss为-6，胜率为0.516741，收益率为0.841914
找到参数组：n为24,score_Bi

找到参数组：n为28,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为6,score_MinorLoss为-10，胜率为0.508929，收益率为0.176233
找到参数组：n为4,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-10，胜率为0.520089，收益率为0.449990
找到参数组：n为8,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-10，胜率为0.513393，收益率为0.750696
找到参数组：n为12,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-10，胜率为0.515625，收益率为1.078455
找到参数组：n为16,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-10，胜率为0.520089，收益率为1.039477
找到参数组：n为20,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-10，胜率为0.517857，收益率为1.451094
找到参数组：n为24,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为8,score_MinorLoss为-10，胜率为0.517857，收益率为0.505925
找到参数组：n为8,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为2,score_MinorLoss为-8，胜率为0.507812，收益率为0.307111
找到参数组：n为12,score_BigBonus为0,score_BigLoss为-8,score_MinorBonus为2,score_MinorLoss为-8，胜率为0.507812，收益率为0.450638
找到参数组：n为16,score_BigBonu

In [ ]:
para_pd = pd.DataFrame(para_list)
para_pd.columns = ["n_windows",'score_BigBonus','score_BigLoss','score_MinorBonus','score_MinorLoss','win_rate','yield_rate']

# write_file('best_para_HMM.csv', para_pd.to_csv(), append=False)
# data_total_pd.to_csv(".\\HMM\\best_para_HMM.csv")
data_total_pd.to_csv("best_para_HMM.csv")